In [1]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

base = np.array([[20],[30],[50],[120],[22],[63],[102],[34],[87],[12],[25],[128],[23],[77],[56]])
target = np.array([[50],[70],[110],[250],[54],[136],[214],[78],[184],[34],[60],[266],[56],[166],[122]])
#mixed = base
#mixed = base.extend(target)
mixed = np.concatenate((base,target))

scaler = MinMaxScaler()
mixed_scaled = scaler.fit_transform(mixed)

base_scaled = mixed_scaled[:15]
#base_unscaled = scaler.inverse_transform(base_scaled)

target_scaled = mixed_scaled[15:20]
#target_unscaled = scaler.inverse_transform(target_scaled)
#print(mixed)
# scaler = MinMaxScaler()
# base_scaled = scaler.fit_transform(base)
# base_unscaled = scaler.inverse_transform(base_scaled)

# target_scaled = scaler.fit_transform(target[:5])
# target_unscaled = scaler.inverse_transform(target_scaled)

num_clusters = 3
num_attr = mixed_scaled[0].size
print(target_scaled)

[[ 0.1496063 ]
 [ 0.22834646]
 [ 0.38582677]
 [ 0.93700787]
 [ 0.16535433]]


/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


In [2]:

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(base_scaled)

base_centroids = kmeans.cluster_centers_
#labels = kmeans.labels_
    
print(kmeans.cluster_centers_)
print(kmeans.inertia_)

[[ 0.41207349]
 [ 0.04611924]
 [ 0.21496063]]
0.0245101252107


In [3]:
def cal_centroids(alpha,beta):

    target_centroids = []
    for i in range(num_clusters):
        val = []
        for j in range(num_attr):
            val.append(alpha[j]*base_centroids[i][j] + beta[j])
        
        target_centroids.append(val)
             

    target_centroids = np.array(target_centroids)
    print("new centroids: ", target_centroids)
    return target_centroids
    #print(target_centroids)

In [4]:
def kmeans(k,dataItems,centroids,maxIter,num_attr):
    
    #print(k,dataItems,centroids,maxInter,num_attr)
    old_centroids = []
    groups = []
    
    for i in range(k):
        groups.append([])
        
    iter = 0
    while(iter < maxIter):
        old_centroids = centroids
        for item in dataItems:
            row = []
            for centroid in centroids:
                diff = 0
                for i in range(num_attr):
                    diff += abs(item[i] - centroid[i])
                    
                row.append(diff)
                
            idx = row.index(min(row))
            groups[idx].append(item)
                    
            
        iter += 1
        
    ss = 0    
    for i in range(k):
        for item in groups[i]:
            for j in range(num_attr):
            
                diff = abs(item[j]- old_centroids[i][j])
                ss += pow(diff,2)
            
        
    return (groups,ss/2)
        

In [5]:
def cal_gradient(clusters,reframed_centroids,old_alpha,old_beta):
    
    gradient_alpha = []
    gradient_beta = []
    gradient = 0
    
    for i in range(num_attr):
        gradient_alpha.append(0)
        gradient_beta.append(0)
        
    
    for i in range(num_clusters):
                
        for member in clusters[i]:
                      
            for j in range(num_attr):
                
                gradient =  (reframed_centroids[i][j] - member[j])
                gradient_alpha[j] = gradient_alpha[j] + gradient*reframed_centroids[i][j]
                gradient_beta[j] = gradient_beta[j] + gradient

    print("******")
    #print(gradient_alpha,gradient_beta)
    
    new_alpha = []
    new_beta = []
    
    for i in range(num_attr):       
        new_alpha.append(old_alpha[i]-.1*gradient_alpha[i])
        new_beta.append(old_beta[i]-.1*gradient_beta[i])
        
    return [new_alpha,new_beta]

In [6]:
alpha = []
beta = []
avg_m = []
avg_d = []
sum_d = 0
for i in range(num_attr):
    sum_m = 0
    for j in range(num_clusters):
        sum_m = sum_m + base_centroids[j][i]
        
    avg_m.append(sum_m/num_clusters)
    

for i in range(num_attr):
    sum_d = 0
    for j in range(len(target_scaled)):
        sum_d = sum_d + target_scaled[j][i]
        
    avg_d.append(sum_d/len(target_scaled))
    

for i in range(num_attr):
    alpha.append(avg_m[i]/avg_d[i])
    beta.append(0)

print(alpha,beta)
        


[0.60119884803429113] [0]


In [7]:
reframed_centroids =  cal_centroids(alpha,beta)
km = kmeans(num_clusters, target_scaled, reframed_centroids, 1, 1)
#kmeans.fit(target_scaled)
best_error = km[1]
#centroids = kmeans.cluster_centers_
#labels = kmeans.labels_

count = 0
best_alpha = alpha
best_beta = beta
while(1):
    #print(centroids)
    print("best error: ",best_error)
    
    #clusters = find_members(centroids,labels)
    
    #reframed_centroids = closest_centroids(reframed_centroids,centroids)
    
    new_alphabeta = cal_gradient(km[0],reframed_centroids,alpha,beta)
    
    alpha = new_alphabeta[0]
    beta = new_alphabeta[1]
    
    print("new alpha beta", alpha, beta)
    
    reframed_centroids =  cal_centroids(alpha,beta)
    
    km = kmeans(num_clusters, target_scaled, reframed_centroids, 1, 1)
    #kmeans.fit(target_scaled)
    new_error = km[1]
    
    print("compare ",best_error,new_error)
    if(new_error < best_error):
        best_alpha = alpha
        best_beta = beta
        best_error = new_error
        count = 0
        
    elif(new_error == best_error):
        if(count<5):
            count += 1
            continue
        else:
            break;
                
    else:
        break; 

    #base_centroids = kmeans.cluster_centers_
    #labels = kmeans.labels_

    #print(kmeans.cluster_centers_)
    #print(old_error,new_error)
    
print("finalparameters", best_alpha,best_beta)

new centroids:  [[ 0.24773811]
 [ 0.02772683]
 [ 0.12923408]]
best error:  0.24812851251
******
new alpha beta [0.62194533930949314] [0.086445924217278744]
new centroids:  [[ 0.34273311]
 [ 0.11512957]
 [ 0.22013969]]
compare  0.24812851251 0.179399040066
best error:  0.179399040066
******
new alpha beta [0.64494589381456169] [0.15947359306857437]
new centroids:  [[ 0.4252387 ]
 [ 0.189218  ]
 [ 0.29811157]]
compare  0.179399040066 0.133565293239
best error:  0.133565293239
******
new alpha beta [0.66457166427947767] [0.20427462520237943]
new centroids:  [[ 0.47812699]
 [ 0.23492416]
 [ 0.34713137]]
compare  0.133565293239 0.112115682269
best error:  0.112115682269
******
new alpha beta [0.68406202431217389] [0.23788571386501806]
new centroids:  [[ 0.51976954]
 [ 0.26943413]
 [ 0.38493212]]
compare  0.112115682269 0.100484069412
best error:  0.100484069412
******
new alpha beta [0.69864336208659994] [0.25319948197081271]
new centroids:  [[ 0.54109189]
 [ 0.28542038]
 [ 0.4033803 ]]
com

compare  0.0249842800297 0.0245483243172
best error:  0.0245483243172
******
new alpha beta [1.4567738341591876] [0.15259428291182453]
new centroids:  [[ 0.75289216]
 [ 0.21977958]
 [ 0.4657433 ]]
compare  0.0245483243172 0.0241225177456
best error:  0.0241225177456
******
new alpha beta [1.4643635641848005] [0.15141103618147844]
new centroids:  [[ 0.75483644]
 [ 0.21894636]
 [ 0.46619155]]
compare  0.0241225177456 0.0237067203113
best error:  0.0237067203113
******
new alpha beta [1.4718822477180757] [0.1502385010839738]
new centroids:  [[ 0.75676216]
 [ 0.21812058]
 [ 0.46663524]]
compare  0.0237067203113 0.0233007898749
best error:  0.0233007898749
******
new alpha beta [1.4793294228992542] [0.14907675964334016]
new centroids:  [[ 0.7586692 ]
 [ 0.2173023 ]
 [ 0.46707434]]
compare  0.0233007898749 0.0229045823636
best error:  0.0229045823636
******
new alpha beta [1.4867046641973074] [0.14792588822196076]
new centroids:  [[ 0.76055747]
 [ 0.21649157]
 [ 0.46750886]]
compare  0.02290

compare  0.00965976398661 0.00965963309786
best error:  0.00965963309786
******
new alpha beta [1.9731846729201703] [0.071224470159054698]
new centroids:  [[ 0.88432157]
 [ 0.16222624]
 [ 0.49538149]]
compare  0.00965963309786 0.0096595072785
best error:  0.0096595072785
******
new alpha beta [1.9733353595332304] [0.071200466319424477]
new centroids:  [[ 0.88435966]
 [ 0.16220918]
 [ 0.49538988]]
compare  0.0096595072785 0.00965938633275
best error:  0.00965938633275
******
new alpha beta [1.9734831042071692] [0.071176930969416657]
new centroids:  [[ 0.884397  ]
 [ 0.16219246]
 [ 0.4953981 ]]
compare  0.00965938633275 0.00965927007237
best error:  0.00965927007237
******
new alpha beta [1.9736279639424332] [0.071153855037835625]
new centroids:  [[ 0.88443362]
 [ 0.16217607]
 [ 0.49540617]]
compare  0.00965927007237 0.00965915831636
best error:  0.00965915831636
******
new alpha beta [1.9737699946520308] [0.071131229626316045]
new centroids:  [[ 0.88446952]
 [ 0.16215999]
 [ 0.49541407]

best error:  0.0096563931831
******
new alpha beta [1.9806780092140293] [0.070030620220114512]
new centroids:  [[ 0.88621552]
 [ 0.16137797]
 [ 0.49579841]]
compare  0.0096563931831 0.00965639310146
best error:  0.00965639310146
******
new alpha beta [1.9806818543085576] [0.070030007515059195]
new centroids:  [[ 0.88621649]
 [ 0.16137754]
 [ 0.49579863]]
compare  0.00965639310146 0.00965639302301
best error:  0.00965639302301
******
new alpha beta [1.9806856237680355] [0.070029406862139335]
new centroids:  [[ 0.88621745]
 [ 0.16137711]
 [ 0.49579884]]
compare  0.00965639302301 0.00965639294761
best error:  0.00965639294761
******
new alpha beta [1.9806893190799604] [0.070028818024332098]
new centroids:  [[ 0.88621838]
 [ 0.16137669]
 [ 0.49579904]]
compare  0.00965639294761 0.00965639287515
best error:  0.00965639287515
******
new alpha beta [1.9806929417025863] [0.070028240769274289]
new centroids:  [[ 0.8862193 ]
 [ 0.16137628]
 [ 0.49579924]]
compare  0.00965639287515 0.009656392805

new centroids:  [[ 0.88626342]
 [ 0.16135651]
 [ 0.49580895]]
compare  0.0096563910896 0.00965639108953
best error:  0.00965639108953
******
new alpha beta [1.9808676623327339] [0.070000399405637487]
new centroids:  [[ 0.88626345]
 [ 0.1613565 ]
 [ 0.49580896]]
compare  0.00965639108953 0.00965639108946
best error:  0.00965639108946
******
new alpha beta [1.9808677765007481] [0.070000381213137078]
new centroids:  [[ 0.88626348]
 [ 0.16135649]
 [ 0.49580897]]
compare  0.00965639108946 0.00965639108939
best error:  0.00965639108939
******
new alpha beta [1.9808678884225923] [0.070000363378560387]
new centroids:  [[ 0.88626351]
 [ 0.16135648]
 [ 0.49580897]]
compare  0.00965639108939 0.00965639108932
best error:  0.00965639108932
******
new alpha beta [1.9808679981424577] [0.070000345894865576]
new centroids:  [[ 0.88626354]
 [ 0.16135646]
 [ 0.49580898]]
compare  0.00965639108932 0.00965639108926
best error:  0.00965639108926
******
new alpha beta [1.9808681057036666] [0.0700003287551493

compare  0.00965639108768 0.00965639108768
best error:  0.00965639108768
******
new alpha beta [1.9808733975288597] [0.069999485510697346]
new centroids:  [[ 0.8862649 ]
 [ 0.16135585]
 [ 0.49580928]]
compare  0.00965639108768 0.00965639108768
best error:  0.00965639108768
******
new alpha beta [1.98087339886087] [0.069999485298443478]
new centroids:  [[ 0.8862649 ]
 [ 0.16135585]
 [ 0.49580928]]
compare  0.00965639108768 0.00965639108768
best error:  0.00965639108768
******
new alpha beta [1.9808734001666737] [0.069999485090365562]
new centroids:  [[ 0.8862649 ]
 [ 0.16135585]
 [ 0.49580928]]
compare  0.00965639108768 0.00965639108768
best error:  0.00965639108768
******
new alpha beta [1.9808734014467866] [0.069999484886381469]
new centroids:  [[ 0.8862649 ]
 [ 0.16135585]
 [ 0.49580928]]
compare  0.00965639108768 0.00965639108768
best error:  0.00965639108768
******
new alpha beta [1.9808734027017141] [0.069999484686410637]
new centroids:  [[ 0.8862649 ]
 [ 0.16135585]
 [ 0.49580928

In [8]:
target_cent = cal_centroids(best_alpha,best_beta)

new centroids:  [[ 0.88626491]
 [ 0.16135585]
 [ 0.49580928]]


In [9]:
target_sc = mixed_scaled[15:]
# base_unscaled = scaler.inverse_transform(base_scaled)
# target_unscaled = scaler.inverse_transform(target_sc)
# print(target_sc)
# print(base_scaled)
# print(base_unscaled)
# print(target_unscaled)

In [10]:
kmeans = KMeans(n_clusters=num_clusters, init=target_cent, max_iter=1)
kmeans.fit(target_sc)
print(kmeans.inertia_)

0.0993797035213


/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


In [11]:
kmeans = KMeans(n_clusters=num_clusters, init=base_centroids, max_iter=1)
kmeans.fit(target_sc)
print(kmeans.inertia_)

0.337106729769


/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
